In [1]:
# pip install psycopg2-binary
import psycopg2

# Replace these placeholders with your actual database information
db_host = "34.132.74.124"
db_name = "dvdrental"
db_user = "postgres"
db_password = "DataBootcamp2023"

# PostgreSQL connection parameters
conn_params = {
    "host": "localhost",
    "port": "5433",
    "database": "deb_captone_project",
    "user": "postgres",
    "password": "postgres"
}

# Schema and table creation SQL statements
schema_name = "capstone_schema_name"
create_schema_query = f"CREATE SCHEMA IF NOT EXISTS {schema_name};"
create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {schema_name}.user_purchase (
        invoice_number VARCHAR(10),
        stock_code VARCHAR(20),
        detail VARCHAR(1000),
        quantity INT,
        invoice_date TIMESTAMP,
        unit_price NUMERIC(8,3),
        customer_id INT,
        country VARCHAR(20)
    );
"""

def create_user_purchase_table(conn, cursor):
    # Create schema
    cursor.execute(create_schema_query)
    conn.commit()
    
    # Create table
    cursor.execute(create_table_query)
    conn.commit()
    print("Table 'user_purchase' created successfully.")

try:
    # Connect to PostgreSQL
    connection = psycopg2.connect(**conn_params)
    cursor = connection.cursor()

    # Create user_purchase table
    create_user_purchase_table(connection, cursor)

except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL or executing SQL:", error)
finally:
    # Close the cursor and connection
    if cursor:
        cursor.close()
    if connection:
        connection.close()


Table 'user_purchase' created successfully.


In [1]:
import psycopg2
import csv
from datetime import datetime
from pathlib import Path

# CSV file path
csv_file_path = Path('data/user_purchase.csv')

# Replace these placeholders with your actual database information
db_host = "34.132.74.124"
db_name = "dvdrental"
db_user = "postgres"
db_password = "DataBootcamp2023"

# PostgreSQL connection parameters
conn_params = {
    "host": "localhost",
    "port": "5433",
    "database": "deb_captone_project",
    "user": "postgres",
    "password": "postgres"
}

# Schema and table creation SQL statements
schema_name = "capstone"

def read_csv_and_insert_data():
    try:
        # Connect to PostgreSQL
        # print(conn_params)
        connection = psycopg2.connect(**conn_params)
        cursor = connection.cursor()
        # print(cursor)

        with open(csv_file_path, 'r') as csvfile:
            csv_reader = csv.reader(csvfile)
            next(csv_reader)  # Skip header

            for row in csv_reader:
                # print (row)
                invoice_number, stock_code, detail, quantity, invoice_date, unit_price, customer_id, country = row

                # Convert invoice_date to datetime
                # invoice_date = invoice_date.replace("/", "-") 
              
                invoice_date = datetime.strptime(invoice_date, '%m/%d/%Y %H:%M') #12/1/2010 8:26
                invoice_date = datetime.strftime(invoice_date, '%Y-%m-%d %H:%M')
               
                if(customer_id) :
                    # SQL query to insert data into the user_purchase table
                    insert_query = f"""
                        INSERT INTO {schema_name}.user_purchase
                        (invoice_number, stock_code, detail, quantity, invoice_date, unit_price, customer_id, country)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
                    """

                    # Execute the insert query
                    cursor.execute(insert_query, (invoice_number, stock_code, detail, quantity, invoice_date, unit_price, customer_id, country))

        # Commit the transaction
        connection.commit()
        print("Data imported successfully.")

    except (Exception, psycopg2.Error) as error:
        print("Error while connecting to PostgreSQL or inserting data:", error)
    finally:
        # Close the cursor and connection
        if cursor:
            cursor.close()
        if connection:
            connection.close()

# Call the function to import data from the CSV file
read_csv_and_insert_data()


Data imported successfully.


In [4]:
import pandas as pd
df = pd.read_csv('data/user_purchase.csv')

df = df.dropna()  # Remove null values
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], format='%m/%d/%Y %H:%M').dt.strftime('%Y-%m-%d %H:%M')
df.head()
 # Store the cleaned data back to a CSV file (you can modify this to store in a different format)
df.to_csv('data/cleaned_user_purchase.csv', index=False)

In [8]:

df_cleaned = pd.read_csv('data/cleaned_user_purchase.csv')
print('cleaned data ',df_cleaned.shape)    
print('raw data ',df.shape)
df_cleaned.isnull().sum()
df.isnull().sum()

cleaned data  (406828, 8)
raw data  (406828, 8)


InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [35]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.sql.functions import current_timestamp, col, when

# Initialize a Spark session
spark = SparkSession.builder \
    .appName("MovieReviewAnalysis") \
    .getOrCreate()

# Load the movie_review.csv into a DataFrame
csv_file_path = Path('data/movie_review.csv')
output_csv_file_path = Path('data/staging_area/movie_reviewed.csv')
df = spark.read.csv(csv_file_path , header=True, inferSchema=True)

# Tokenize the review_str column
tokenizer = Tokenizer(inputCol="review_str", outputCol="review_token")
df = tokenizer.transform(df)

# Remove stop words
remover = StopWordsRemover(inputCol="review_token", outputCol="filtered_tokens")
df = remover.transform(df)

# Check if the review contains the word "good" and label as positive review
df = df.withColumn("positive_review", when(col("review_str").contains("good"), 1).otherwise(0))

# Add a timestamp column for insert_date
df = df.withColumn("insert_date", current_timestamp())

# Select the required columns
result_df = df.select("cid", "positive_review", "review_id")

# Save the results to a new file in the STAGE_area folder
result_df.write.mode("overwrite").csv(output_csv_file_path, header=True)

# Stop the Spark session
spark.stop()


ModuleNotFoundError: No module named 'pyspark'